The likelyhood of getting diabetes is altered by a number of factors including:

- Age
- Ethnicity
- Family History
- Weight
- Physical Inactivity
- Genetics
- Other Conditions: High blood pressure, high cholesterol, and heart disease increase the risk

To explore this data I will be using synthetic patients records from [SynthericMass](https://synthea.mitre.org/) using a community version of a FHIR enabled Data Platform from Intersystems called IRIS [Intersystems Open Exchange - iris-fhir-template](https://openexchange.intersystems.com/package/iris-fhir-template).
These examples are based on US population demographics and will also use a mix of LOINC and SNOMED coding, so codes used are likely to differ from UK based systems which predominantly use SNOMED CT UK Edition.

In [1]:
import iris

host = "localhost"
# this is the superserver port
port = 32782
namespace = "FHIRSERVER"
user = "_SYSTEM"
password = "SYS"

conn = iris.connect(
    hostname=host,
    port=port,
    namespace=namespace,
    username=user,
    password=password
)

# create a cursor
cur = conn.cursor()

sql = """
      select patient._id, patient.KEY, DATEDIFF('year',birthdate,$HOROLOG) as AGE, gender,
             IFNULL(diabetes._id,0,1) as TYPE2DIABETES,
             IFNULL(hypertension._id,0,1) as HYPERTENSION
      from HSFHIR_X0001_S.Patient patient
               left outer join HSFHIR_X0001_S.Condition diabetes on patient.Key = diabetes.patient and diabetes.code [ '44054006'
               left outer join HSFHIR_X0001_S.Condition hypertension on patient.Key = hypertension.patient and hypertension.code [ '59621000'
      """
cur.execute(sql)
result = cur.fetchall()
for row in result:
    patient = row[0]
    print('patient id = ',row[0])

patient id =  4
patient id =  2
patient id =  3
patient id =  5
patient id =  6
patient id =  38
patient id =  2380
patient id =  2371
patient id =  2376
patient id =  2372
patient id =  2374
patient id =  2382
patient id =  2375
patient id =  2379
patient id =  2383
patient id =  2377
patient id =  2378
patient id =  2370
patient id =  2373
patient id =  2381
patient id =  2369
patient id =  2429
patient id =  3876
patient id =  3443
patient id =  5037
patient id =  5323
patient id =  5475
patient id =  6268
patient id =  6338
patient id =  7882
patient id =  6942
patient id =  8374
patient id =  9100
patient id =  9218
patient id =  9303
patient id =  8797
patient id =  9776
patient id =  10042
patient id =  10648
patient id =  10816
patient id =  10214
patient id =  10588
patient id =  11815
patient id =  11146
patient id =  12069
patient id =  12189
patient id =  12318
patient id =  13165
patient id =  14073
patient id =  10897
patient id =  13783
patient id =  15192
patient id =  